<a href="https://colab.research.google.com/github/kridtapon/Candlestick-Pattern/blob/main/Candlestick_Pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
pip install mplfinance

In [19]:
import pandas as pd
import numpy as np
import yfinance as yf
import mplfinance as mpf

In [50]:
# Download the data
df = yf.download("BTC-USD", start="2014-11-29", end="2024-11-29", interval="1d")

[*********************100%***********************]  1 of 1 completed


In [15]:
#df = pd.read_csv('GBPUSD_2023_H1.csv')

In [51]:
data_clean = df.copy()
data_clean = data_clean.ffill()
data_clean.columns = ['Date','Close','High','Low','Open','Volume']
data_clean
data_clean.drop(['Date'], axis='columns', inplace=True)

In [31]:
data = data_clean.copy()
data

,Close,High,Low,Open,Volume
Date,,,,,
2014-11-29,375.490997,387.601013,372.144989,376.152008,15375600
2014-11-30,378.046997,382.527008,373.308990,375.510010,9194440
2014-12-01,379.244995,383.661987,376.669006,378.248993,11763000
2014-12-02,381.315002,384.037994,377.863007,379.250000,12364100
2014-12-03,375.010010,383.026001,374.346008,381.721985,13340100
...,...,...,...,...,...
2024-11-24,98013.820312,98647.179688,95788.078125,97778.093750,51712020623
2024-11-25,93102.296875,98935.031250,92642.914062,98033.445312,80909462490
2024-11-26,91985.320312,94991.750000,90770.812500,93087.281250,91656519855


In [35]:
def detect_bullish_patterns(data):
    """
    Detect bullish candlestick patterns in the OHLC data.
    :param data: DataFrame with columns ['Open', 'High', 'Low', 'Close']
    :return: Dictionary with patterns and their respective boolean series
    """
    patterns = {}

    # Hammer
    patterns['Hammer'] = (
        (data['High'] - data['Low'] > 3 * (data['Open'] - data['Close'])) &  # Long lower wick
        ((data['Close'] - data['Low']) / (0.001 + data['High'] - data['Low']) > 0.6) &  # Close near the high
        ((data['Open'] - data['Low']) / (0.001 + data['High'] - data['Low']) > 0.6)  # Open near the high
    )

    # Inverted Hammer
    patterns['Inverted Hammer'] = (
        (data['High'] - data['Low'] > 3 * (data['Open'] - data['Close'])) &  # Long upper wick
        ((data['High'] - data['Close']) / (0.001 + data['High'] - data['Low']) > 0.6) &  # Close near the low
        ((data['High'] - data['Open']) / (0.001 + data['High'] - data['Low']) > 0.6)  # Open near the low
    )

    # Bullish Engulfing
    patterns['Bullish Engulfing'] = (
        (data['Close'].shift(1) < data['Open'].shift(1)) &  # Previous red candle
        (data['Close'] > data['Open']) &  # Current green candle
        (data['Close'] > data['Open'].shift(1)) &  # Current close above previous open
        (data['Open'] < data['Close'].shift(1))  # Current open below previous close
    )

    # Morning Star
    patterns['Morning Star'] = (
        (data['Close'].shift(2) < data['Open'].shift(2)) &  # Large red candle
        (np.abs(data['Close'].shift(1) - data['Open'].shift(1)) / (data['High'].shift(1) - data['Low'].shift(1)) < 0.2) &  # Small-bodied candle
        (data['Close'] > data['Open'])  # Large green candle
    )

    # Piercing Line
    patterns['Piercing Line'] = (
        (data['Close'].shift(1) < data['Open'].shift(1)) &  # Previous red candle
        (data['Open'] < data['Close'].shift(1)) &  # Current open below previous close
        (data['Close'] > (data['Open'].shift(1) + data['Close'].shift(1)) / 2)  # Current close above midpoint
    )

    # Three White Soldiers
    patterns['Three White Soldiers'] = (
        (data['Close'] > data['Open']) &  # Current green candle
        (data['Close'].shift(1) > data['Open'].shift(1)) &  # Previous green candle
        (data['Close'].shift(2) > data['Open'].shift(2)) &  # Candle before previous
        (data['Open'] > data['Close'].shift(1)) &  # Open within the body of the previous candle
        (data['Open'].shift(1) > data['Close'].shift(2))  # Previous open within the body of the candle before previous
    )

    return patterns


# Example: Detecting Patterns in Sample Data
if __name__ == "__main__":
    # Load sample data
    # Replace this with your actual data file
    data = data

    # Detect bullish patterns
    bullish_patterns = detect_bullish_patterns(data)

    # Display results
    for pattern, condition in bullish_patterns.items():
        detected_days = data[condition].index.tolist()
        print(f"{pattern} detected on days: {detected_days}\n")


Hammer detected on days: [Timestamp('2015-01-20 00:00:00'), Timestamp('2015-01-23 00:00:00'), Timestamp('2015-01-29 00:00:00'), Timestamp('2015-02-04 00:00:00'), Timestamp('2015-02-10 00:00:00'), Timestamp('2015-02-24 00:00:00'), Timestamp('2015-02-26 00:00:00'), Timestamp('2015-02-28 00:00:00'), Timestamp('2015-03-21 00:00:00'), Timestamp('2015-03-25 00:00:00'), Timestamp('2015-04-04 00:00:00'), Timestamp('2015-04-09 00:00:00'), Timestamp('2015-04-12 00:00:00'), Timestamp('2015-05-29 00:00:00'), Timestamp('2015-06-18 00:00:00'), Timestamp('2015-06-20 00:00:00'), Timestamp('2015-07-17 00:00:00'), Timestamp('2015-08-22 00:00:00'), Timestamp('2015-10-26 00:00:00'), Timestamp('2015-11-13 00:00:00'), Timestamp('2015-11-18 00:00:00'), Timestamp('2015-11-20 00:00:00'), Timestamp('2015-11-28 00:00:00'), Timestamp('2015-12-02 00:00:00'), Timestamp('2015-12-04 00:00:00'), Timestamp('2015-12-17 00:00:00'), Timestamp('2015-12-19 00:00:00'), Timestamp('2015-12-21 00:00:00'), Timestamp('2016-01-10 

In [32]:
def detect_bearish_patterns(data):
    """
    Detect bearish candlestick patterns in the OHLC data.
    :param data: DataFrame with columns ['Open', 'High', 'Low', 'Close']
    :return: Dictionary with patterns and their respective boolean series
    """
    patterns = {}

    # Hanging Man
    patterns['Hanging Man'] = (
        (data['High'] - data['Low'] > 3 * (data['Open'] - data['Close'])) &  # Long lower wick
        ((data['Close'] - data['Low']) / (0.001 + data['High'] - data['Low']) < 0.4) &  # Close near the low
        ((data['Open'] - data['Low']) / (0.001 + data['High'] - data['Low']) < 0.4)  # Open near the low
    )

    # Shooting Star
    patterns['Shooting Star'] = (
        (data['High'] - data['Low'] > 3 * (data['Open'] - data['Close'])) &  # Long upper wick
        ((data['High'] - data['Close']) / (0.001 + data['High'] - data['Low']) > 0.6) &  # Close near the low
        ((data['High'] - data['Open']) / (0.001 + data['High'] - data['Low']) > 0.6)  # Open near the low
    )

    # Bearish Engulfing
    patterns['Bearish Engulfing'] = (
        (data['Close'].shift(1) > data['Open'].shift(1)) &  # Previous green candle
        (data['Close'] < data['Open']) &  # Current red candle
        (data['Open'] > data['Close'].shift(1)) &  # Current open above previous close
        (data['Close'] < data['Open'].shift(1))  # Current close below previous open
    )

    # Evening Star
    patterns['Evening Star'] = (
        (data['Close'].shift(2) > data['Open'].shift(2)) &  # Large green candle
        (np.abs(data['Close'].shift(1) - data['Open'].shift(1)) / (data['High'].shift(1) - data['Low'].shift(1)) < 0.2) &  # Small-bodied candle
        (data['Close'] < data['Open'])  # Large red candle
    )

    # Dark Cloud Cover
    patterns['Dark Cloud Cover'] = (
        (data['Close'].shift(1) > data['Open'].shift(1)) &  # Previous green candle
        (data['Open'] > data['Close'].shift(1)) &  # Current open above previous close
        (data['Close'] < (data['Open'].shift(1) + data['Close'].shift(1)) / 2)  # Current close below midpoint
    )

    # Three Black Crows
    patterns['Three Black Crows'] = (
        (data['Close'] < data['Open']) &  # Current red candle
        (data['Close'].shift(1) < data['Open'].shift(1)) &  # Previous red candle
        (data['Close'].shift(2) < data['Open'].shift(2)) &  # Candle before previous
        (data['Open'] < data['Close'].shift(1)) &  # Open within the body of the previous candle
        (data['Open'].shift(1) < data['Close'].shift(2))  # Previous open within the body of the candle before previous
    )

    return patterns


# Example: Detecting Patterns in Sample Data
if __name__ == "__main__":
    # Load sample data
    # Replace this with your actual data file
    data = data

    # Detect bearish patterns
    bearish_patterns = detect_bearish_patterns(data)

    # Display results
    for pattern, condition in bearish_patterns.items():
        detected_days = data[condition].index.tolist()
        print(f"{pattern} detected on days: {detected_days}\n")

Hanging Man detected on days: [Timestamp('2014-11-29 00:00:00'), Timestamp('2014-12-01 00:00:00'), Timestamp('2015-01-12 00:00:00'), Timestamp('2015-01-26 00:00:00'), Timestamp('2015-02-11 00:00:00'), Timestamp('2015-02-21 00:00:00'), Timestamp('2015-03-10 00:00:00'), Timestamp('2015-03-23 00:00:00'), Timestamp('2015-03-26 00:00:00'), Timestamp('2015-03-27 00:00:00'), Timestamp('2015-04-19 00:00:00'), Timestamp('2015-05-17 00:00:00'), Timestamp('2015-05-27 00:00:00'), Timestamp('2015-06-09 00:00:00'), Timestamp('2015-06-12 00:00:00'), Timestamp('2015-07-06 00:00:00'), Timestamp('2015-07-15 00:00:00'), Timestamp('2015-07-28 00:00:00'), Timestamp('2015-08-19 00:00:00'), Timestamp('2015-09-07 00:00:00'), Timestamp('2015-09-15 00:00:00'), Timestamp('2015-09-20 00:00:00'), Timestamp('2015-09-25 00:00:00'), Timestamp('2015-10-02 00:00:00'), Timestamp('2015-10-25 00:00:00'), Timestamp('2015-11-04 00:00:00'), Timestamp('2015-12-03 00:00:00'), Timestamp('2015-12-06 00:00:00'), Timestamp('2015-1

In [48]:
def detect_neutral_patterns(data):
    """
    Detect neutral/indecisive candlestick patterns in the OHLC data.
    :param data: DataFrame with columns ['Open', 'High', 'Low', 'Close']
    :return: Dictionary with patterns and their respective boolean series
    """
    patterns = {}

    # Doji (Standard)
    patterns['Doji'] = (
        np.abs(data['Close'] - data['Open']) / (data['High'] - data['Low']) < 0.1  # Small body
    )

    # Dragonfly Doji
    #patterns['Dragonfly Doji'] = (
      #  np.abs(data['Close'] - data['Open']) / (data['High'] - data['Low']) < 0.1 &  # Small body
      #  (data['High'] - np.maximum(data['Open'], data['Close'])) / (data['High'] - data['Low']) < 0.1  # No upper wick
   # )

    # Gravestone Doji
  #  patterns['Gravestone Doji'] = (
    #    np.abs(data['Close'] - data['Open']) / (data['High'] - data['Low']) < 0.1 &  # Small body
   #     (np.minimum(data['Open'], data['Close']) - data['Low']) / (data['High'] - data['Low']) < 0.1  # No lower wick
  #  )

    # Spinning Top
    patterns['Spinning Top'] = (
        (np.abs(data['Close'] - data['Open']) / (data['High'] - data['Low']) > 0.1) &  # Small body
        (np.abs(data['High'] - data['Close']) / (data['High'] - data['Low']) < 0.5) &  # Upper wick less than half of range
        (np.abs(data['Open'] - data['Low']) / (data['High'] - data['Low']) < 0.5)  # Lower wick less than half of range
    )

    # Bullish Harami
    patterns['Bullish Harami'] = (
        (data['Close'] > data['Open']) &  # Current green candle
        (data['Close'].shift(1) < data['Open'].shift(1)) &  # Previous red candle
        (data['Open'] > data['Close'].shift(1)) &  # Current open within previous candle
        (data['Close'] < data['Open'].shift(1))  # Current close within previous candle
    )

    # Bearish Harami
    patterns['Bearish Harami'] = (
        (data['Close'] < data['Open']) &  # Current red candle
        (data['Close'].shift(1) > data['Open'].shift(1)) &  # Previous green candle
        (data['Open'] < data['Close'].shift(1)) &  # Current open within previous candle
        (data['Close'] > data['Open'].shift(1))  # Current close within previous candle
    )

    return patterns


# Example: Detecting Patterns in Sample Data
if __name__ == "__main__":
    # Load sample data
    # Replace this with your actual data file
    data = data

    # Detect neutral patterns
    neutral_patterns = detect_neutral_patterns(data)

    # Display results
    for pattern, condition in neutral_patterns.items():
        detected_days = data[condition].index.tolist()
        print(f"{pattern} detected on days: {detected_days}\n")

Doji detected on days: [Timestamp('2014-11-29 00:00:00'), Timestamp('2014-12-07 00:00:00'), Timestamp('2015-01-16 00:00:00'), Timestamp('2015-01-23 00:00:00'), Timestamp('2015-01-29 00:00:00'), Timestamp('2015-02-04 00:00:00'), Timestamp('2015-02-10 00:00:00'), Timestamp('2015-02-16 00:00:00'), Timestamp('2015-02-21 00:00:00'), Timestamp('2015-02-24 00:00:00'), Timestamp('2015-03-23 00:00:00'), Timestamp('2015-04-11 00:00:00'), Timestamp('2015-04-12 00:00:00'), Timestamp('2015-04-29 00:00:00'), Timestamp('2015-05-26 00:00:00'), Timestamp('2015-06-03 00:00:00'), Timestamp('2015-06-18 00:00:00'), Timestamp('2015-07-27 00:00:00'), Timestamp('2015-08-19 00:00:00'), Timestamp('2015-09-07 00:00:00'), Timestamp('2015-09-10 00:00:00'), Timestamp('2015-09-14 00:00:00'), Timestamp('2015-09-15 00:00:00'), Timestamp('2015-10-02 00:00:00'), Timestamp('2015-11-04 00:00:00'), Timestamp('2015-11-18 00:00:00'), Timestamp('2015-11-28 00:00:00'), Timestamp('2015-12-06 00:00:00'), Timestamp('2015-12-28 00

In [49]:
def detect_complex_patterns(data):
    """
    Detect complex candlestick patterns in the OHLC data.
    :param data: DataFrame with columns ['Open', 'High', 'Low', 'Close']
    :return: Dictionary with patterns and their respective boolean series
    """
    patterns = {}

    # Bullish Marubozu
    patterns['Bullish Marubozu'] = (
        (data['Close'] > data['Open']) &  # Green candle
        ((data['High'] == data['Close']) & (data['Low'] == data['Open']))  # No wicks
    )

    # Bearish Marubozu
    patterns['Bearish Marubozu'] = (
        (data['Close'] < data['Open']) &  # Red candle
        ((data['High'] == data['Open']) & (data['Low'] == data['Close']))  # No wicks
    )

    # Rising Three Methods
    patterns['Rising Three Methods'] = (
        (data['Close'] > data['Open']) &  # First long green candle
        (data['Close'].shift(1) < data['Open'].shift(1)) &  # First small red candle
        (data['Close'].shift(2) < data['Open'].shift(2)) &  # Second small red candle
        (data['Close'].shift(3) < data['Open'].shift(3)) &  # Third small red candle
        (data['Close'].shift(4) > data['Open'].shift(4)) &  # Last large green candle
        (data['Close'].shift(1) > data['Low']) & (data['Open'].shift(1) < data['Close']) &  # Red candles within range
        (data['Close'].shift(2) > data['Low']) & (data['Open'].shift(2) < data['Close']) &
        (data['Close'].shift(3) > data['Low']) & (data['Open'].shift(3) < data['Close'])
    )

    # Falling Three Methods
    patterns['Falling Three Methods'] = (
        (data['Close'] < data['Open']) &  # First long red candle
        (data['Close'].shift(1) > data['Open'].shift(1)) &  # First small green candle
        (data['Close'].shift(2) > data['Open'].shift(2)) &  # Second small green candle
        (data['Close'].shift(3) > data['Open'].shift(3)) &  # Third small green candle
        (data['Close'].shift(4) < data['Open'].shift(4)) &  # Last large red candle
        (data['Close'].shift(1) < data['High']) & (data['Open'].shift(1) > data['Low']) &  # Green candles within range
        (data['Close'].shift(2) < data['High']) & (data['Open'].shift(2) > data['Low']) &
        (data['Close'].shift(3) < data['High']) & (data['Open'].shift(3) > data['Low'])
    )

    return patterns


# Example: Detecting Patterns in Sample Data
if __name__ == "__main__":
    # Load sample data
    # Replace this with your actual data file
    data = data

    # Detect complex patterns
    complex_patterns = detect_complex_patterns(data)

    # Display results
    for pattern, condition in complex_patterns.items():
        detected_days = data[condition].index.tolist()
        print(f"{pattern} detected on days: {detected_days}\n")

Bullish Marubozu detected on days: [Timestamp('2015-12-11 00:00:00'), Timestamp('2015-12-15 00:00:00'), Timestamp('2016-02-13 00:00:00'), Timestamp('2016-02-14 00:00:00'), Timestamp('2016-12-28 00:00:00'), Timestamp('2017-05-08 00:00:00'), Timestamp('2017-11-27 00:00:00'), Timestamp('2018-07-16 00:00:00'), Timestamp('2019-06-21 00:00:00'), Timestamp('2019-06-25 00:00:00'), Timestamp('2020-04-27 00:00:00')]

Bearish Marubozu detected on days: [Timestamp('2016-01-12 00:00:00'), Timestamp('2016-01-15 00:00:00')]

Rising Three Methods detected on days: [Timestamp('2015-11-25 00:00:00'), Timestamp('2019-05-19 00:00:00'), Timestamp('2019-10-25 00:00:00'), Timestamp('2020-02-05 00:00:00'), Timestamp('2020-04-16 00:00:00'), Timestamp('2021-08-13 00:00:00'), Timestamp('2021-11-02 00:00:00'), Timestamp('2022-10-03 00:00:00'), Timestamp('2023-08-07 00:00:00'), Timestamp('2023-09-07 00:00:00'), Timestamp('2024-01-08 00:00:00')]

Falling Three Methods detected on days: [Timestamp('2014-12-03 00:00: